## Imports

In [1]:
import pandapower as pp
import pandapower.networks as nw
from copy import deepcopy
import pandapower.plotting as plot
import numpy as np
import pandas as pd
import os
import numba

import julia
julia.install()

from julia.api import Julia
jl = Julia(compiled_modules=False)

red = "30" # o "118"


if red == "30":
  net = pp.networks.case30()
else:
  net = pp.networks.case118()

[ Info: Julia version info


Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
      "Manjaro Linux"
  uname: Linux 6.6.19-1-MANJARO #1 SMP PREEMPT_DYNAMIC Fri Mar  1 18:16:16 UTC 2024 x86_64 unknown
  CPU: 13th Gen Intel(R) Core(TM) i5-13400F: 
                 speed         user         nice          sys         idle          irq
       #1-16  3600 MHz    2559891 s        110 s    3792415 s   53635446 s      62891 s
       
  Memory: 62.63359832763672 GB (1848.05859375 MB free)
  Uptime: 377113.0 sec
  Load Avg:  18.83  18.72  18.68
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, goldmont)
Environment:
  HOME = /home/iboero
  PATH = /home/iboero/miniconda3/envs/proy/bin:/home/iboero/miniconda3/condabin:/home/iboero/.vscode-server/cli/servers/Stable-863d2581ecda6849923a2118d93a088b0745d9d6/server/bin/remote-cli:/home/iboero/julia/bin:/home/iboero/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/bin:/opt/cuda/bin:/opt/cuda/nsight_compute

[ Info: Julia executable: /home/iboero/julia/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /home/iboero/miniconda3/envs/proy/bin/python
│     libpython: /home/iboero/miniconda3/envs/proy/lib/libpython3.11.so.1.0
│ Python:
│     python: /home/iboero/miniconda3/envs/proy/bin/python
└     libpython: 
/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj

In [4]:
np.load('./data/input'+red+'_2_0.npy').shape

(5, 30, 4)

In [2]:
# Levantar red


# Permitir solo a los generadores cambiar su potencia al resolver opf
net.load['controllable'] = False
net.gen['controllable'] = True

# Empezar a generar datos
load_p_nom = net.load['p_mw']
load_q_nom = net.load['q_mvar']

X = []
Y = []
X_LOAD = []
# Dividir en 10 batches de 1000 datos
for i in range(10000):
    y = np.zeros((len(net.bus["name"]),2))
    X_load = np.zeros((len(net.bus["name"]),2))
    print('iteracion:', i)
    # Tomar uniforme al rededor del valor nominal
    uniforme = np.random.uniform(0.9,1.1,size=len(load_p_nom))
    uniforme_react = np.random.uniform(0.9,1.1,size=len(load_q_nom))
    net.load['p_mw'] = uniforme*load_p_nom
    net.load['q_mvar'] = uniforme_react*load_q_nom

    # Resolver Flujo de carga
    pp.runpp(net,numba=False)
    X_i = np.array(net.res_bus.values)



    # Resolver OPF
    try:
      pp.runpm_ac_opf(net)
      y[net.gen["bus"]] = np.array([net.res_gen['p_mw'],net.res_gen['q_mvar']]).T
      y[net.ext_grid["bus"]] = np.array([net.res_ext_grid['p_mw'],net.res_ext_grid['q_mvar']]).T
      X_load[net.load["bus"]] = np.array([net.load['p_mw'],net.load['q_mvar']]).T
      Y.append(y)
      X.append(X_i)
      X_LOAD.append(X_load)
    except:
      pass
X = np.array(X)
Y = np.array(Y)
X_LOAD = np.array(X_LOAD)
np.save('./data/input'+red+'_2_0.npy',X)
np.save('./data/p_opt'+red+'_2_0.npy',Y)
np.save('./data/input_load'+red+'_2_0.npy',X_LOAD)

iteracion: 0


[info | PowerModels]: Suppressing information and warning messages for the rest of this session.  Use the Memento package for more fine-grained control of logging.

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

iteracion: 1
iteracion: 2
iteracion: 3
iteracion: 4


OPF did not converge!


iteracion: 5
iteracion: 6


OPF did not converge!


...

iteracion: 3594
iteracion: 3595
iteracion: 3596
iteracion: 3597
iteracion: 3598
iteracion: 3599
iteracion: 3600
iteracion: 3601
iteracion: 3602
iteracion: 3603


OPF did not converge!


iteracion: 3604
iteracion: 3605
iteracion: 3606
iteracion: 3607
iteracion: 3608


OPF did not converge!


iteracion: 3609
iteracion: 3610
iteracion: 3611


In [ ]:
# Levantar red

red = "118" # o "118"


if red == "30":
  net = pp.networks.case30()
else:
  net = pp.networks.case118()

# Permitir solo a los generadores cambiar su potencia al resolver opf
net.load['controllable'] = False
net.gen['controllable'] = True

# Empezar a generar datos
load_p_nom = net.load['p_mw']
load_q_nom = net.load['q_mvar']

X = []
Y = []
X_LOAD = []
# Dividir en 10 batches de 1000 datos
for i in range(10000):
    y = np.zeros((len(net.bus["name"]),2))
    X_load = np.zeros((len(net.bus["name"]),2))
    print('iteracion:', i)
    # Tomar uniforme al rededor del valor nominal
    uniforme = np.random.uniform(0.9,1.1,size=len(load_p_nom))
    uniforme_react = np.random.uniform(0.9,1.1,size=len(load_q_nom))
    net.load['p_mw'] = uniforme*load_p_nom
    net.load['q_mvar'] = uniforme_react*load_q_nom

    # Resolver Flujo de carga
    pp.runpp(net,numba=False)
    X_i = np.array(net.res_bus.values)



    # Resolver OPF
    try:
      pp.runpm_ac_opf(net)
      y[net.gen["bus"]] = np.array([net.res_gen['p_mw'],net.res_gen['q_mvar']]).T
      y[net.ext_grid["bus"]] = np.array([net.res_ext_grid['p_mw'],net.res_ext_grid['q_mvar']]).T
      X_load[net.load["bus"]] = np.array([net.load['p_mw'],net.load['q_mvar']]).T
      Y.append(y)
      X.append(X_i)
      X_LOAD.append(X_load)
    except:
      pass
X = np.array(X)
Y = np.array(Y)
X_LOAD = np.array(X_LOAD)
np.save('./data/input'+red+'_2_0.npy',X)
np.save('./data/p_opt'+red+'_2_0.npy',Y)
np.save('./data/input_load'+red+'_2_0.npy',X_LOAD)

## Resolvemos el OPF de a 1000 datos

In [ ]:
# Levantar red

if red == "30":
  net = pp.networks.case30()
else:
  net = pp.networks.case118()

# Permitir solo a los generadores cambiar su potencia al resolver opf
net.load['controllable'] = False
net.gen['controllable'] = True


# Inicializar df para guardar solucion del opf
columns_load = []
columns_gen = []
columns_ext_grid = []
columns = []

for i in range(len(net.load['p_mw'])):
  columns_load.append(f'load_{i}')
for i in range(len(net.gen['p_mw'])):
  columns_gen.append(f'gen_{i}')
for i in range(len(net.ext_grid['bus'])):
  columns_ext_grid.append(f'ext_grid_{i}')
columns = columns_load + columns_gen + columns_ext_grid

dataset = pd.DataFrame(columns=columns)


# Empezar a generar datos
load_p_nom = net.load['p_mw']
load_q_nom = net.load['q_mvar']

# Dividir en 10 batches de 1000 datos
for j in range(6,10):
  dataset = pd.DataFrame(columns=columns)
  for i in range(1000):
    print('iteracion:', j, '/' , i)
    # Tomar uniforme al rededor del valor nominal
    uniforme = np.random.uniform(0.9,1.1,size=len(load_p_nom))
    uniforme_react = np.random.uniform(0.9,1.1,size=len(load_q_nom))
    net.load['p_mw'] = uniforme*load_p_nom
    net.load['q_mvar'] = uniforme_react*load_q_nom
    # Resolver OPF
    try:
      pp.runpm_ac_opf(net)
      data_i = np.concatenate([net.res_load['p_mw'] + net.res_load['q_mvar']*1j,net.res_gen['p_mw'] + net.res_gen['q_mvar']*1j,net.res_ext_grid['p_mw'] + net.res_ext_grid['q_mvar']*1j]).reshape(1,-1)
      data_i_pd = pd.DataFrame(data_i,columns=columns)
      dataset = pd.concat([dataset,data_i_pd],ignore_index=True,axis=0)
    except:
      pass
  # Al terminar un batch, guardarlo
  dataset.to_csv(f'./data/data_intermedia_' + red + f'/dataset_{j}.csv',index=False)

## Transformamos los datos en un unico numpy

In [12]:
datos_path_list = ['./data/data_intermedia_' + red + '/']

if red=="30":
  net = pp.networks.case30()
else:
  net = pp.networks.case118()

net.load['controllable'] = False
net.gen['controllable'] = True

columns_bus = []
for i in range(len(net.bus['name'])):
  columns_bus.append(f'bus_{i}')

dataset_np_final = []
for datos_path in datos_path_list:
  for f, path in enumerate(os.listdir(datos_path)):
    data = pd.read_csv(datos_path+path)
    for i in data.columns:
      data[i] = data[i].apply(complex)
    zero_data = np.zeros(shape=(data.shape[0],len(columns_bus)))
    dataset_bus = pd.DataFrame(zero_data,columns=columns_bus)
    dataset_np = []

    for i in range(len(net.bus['name'])):
      column_i = np.zeros((dataset_bus.shape[0],4))
      for idx, j in enumerate(net.gen['bus']):
        if i==j:
          column_i[:,0]= np.real(data['gen_'+str(idx)])
          column_i[:,1]= np.imag(data['gen_'+str(idx)])
      for idx, j in enumerate(net.ext_grid['bus']):
        if i==j:
          column_i[:,0]= np.real(data['ext_grid_'+str(idx)])
          column_i[:,1]= np.imag(data['ext_grid_'+str(idx)])
      for idx, j in enumerate(net.load['bus']):
        if i==j:
          column_i[:,2]= np.real(data['load_'+str(idx)])
          column_i[:,3]= np.imag(data['load_'+str(idx)])

      dataset_np.append(column_i)

    dataset_np = np.array(dataset_np).transpose(1,0,2)
    dataset_np_final.append(dataset_np)

dataset_np_final = np.concatenate(dataset_np_final)
np.save('./data/p_opt'+red+'.npy',dataset_np_final)

/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_ax

## Para cada dato, resolvemos el NewtonRaphson y guardamos el estado

In [14]:
datos_path_list = ['./data/data_intermedia_' + red + '/']

X = []
for datos_path in datos_path_list:
  for f, path in enumerate(os.listdir(datos_path)):
    data = pd.read_csv(datos_path+path)
    for i in range(len(data)):
      q_i = data.values[i].astype(complex)[:len(net.load)].imag
      p_i = data.values[i].astype(complex)[:len(net.load)].real
      net.load['p_mw'] = p_i
      net.load['q_mvar'] = q_i
      pp.runpp(net,numba=False)
      X_i = np.array(net.res_bus.values)
      X.append(X_i)
      print(i)
X = np.array(X)

np.save('./data/input'+red+'.npy',X)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27